In [1]:
import tensorflow as tf
import pandas as pd 
import numpy as np
from string import digits
from collections import Counter
from pyvi import ViTokenizer
from gensim.models.word2vec import Word2Vec
from keras.utils.np_utils import to_categorical
import os 
# os.environ["CUDA_VISIBLE_DEVICES"]="7"

%matplotlib inline

2022-05-01 09:15:40.944782: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-01 09:15:40.944814: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
PATH = ""
data_train = pd.read_csv(PATH+"dataset/vlsp_sentiment_train.csv", sep='\t')
data_train.columns =['Class', 'Data']
data_test = pd.read_csv(PATH+"dataset/vlsp_sentiment_test.csv", sep='\t')
data_test.columns =['Class', 'Data']

In [3]:
print(data_train.shape)
print(data_test.shape)

(5100, 2)
(1050, 2)


In [4]:
import bogo
import regex as re
import itertools
def map_to_unicode():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic

def covert_unicode(txt):
    dicchar = map_to_unicode()
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)
  
def preprocess(document):
    if type(document) is not str:
      document = str(document)
    # convert to lower case
    document = document.lower()

    #remove
  
    # remove html character
    document = re.sub(r'<[^>]*>', '', document)

    #convert all char to unicode
    document = covert_unicode(document)

    #remove adjacent identical characters
    document = ''.join(c[0] for c in itertools.groupby(document))

    #uwf=>ừ,....
    document = bogo.process_sequence(document)

    # remove error character
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)

    # remove multiple space character
    document = re.sub(r'\s+', ' ', document).strip()

    return document

In [5]:
data_train['Data'] = data_train['Data'].map(lambda x: preprocess(x))
data_train['Data']

0       mình đã dùng anywhere thế hệ đầu quả là đầy th...
1       quan tâm nhất là độ trễ có cao không dùng thi ...
2       dag xài con cùi bắp 98k pin trâu mỗi tội đánh ...
3       logitech chắc hàng phải tiền triệu trở lên dùn...
4       đang xài con m175 cùi mía nhà xài nhiều chuột ...
                              ...                        
5095    mình mua máy về đc 1 ngày mà điện thoại khác g...
5096    có bạn nào dùng f1ư ko mình dùng m cảm thấy qu...
5097    dùng ôp mà bộ nhớ 4gb thì k chơi games đc đâu ...
5098    sao tui thích xài hàng ôp mà lựa toàn mấy đứa ...
5099    mới mở hộp oy mở vào camera mà đã có ảnh chụp ...
Name: Data, Length: 5100, dtype: object

# Teencode-corrected data

In [6]:
import json
import re
import itertools
import unidecode
import bogo

def read_file(file_path):
    fi = open(file_path, 'r', encoding='utf-8')
    ls = fi.readlines()
    return ls

'''load dictionary to mapping word'''
vowel_file = open(PATH+'correct_teencode_stopword/vietnamese_vowel.json', encoding='utf-8')
vowel_dic = json.load(vowel_file)

short_word_file = open(PATH+'correct_teencode_stopword/short_word.json', encoding='utf-8')
short_word_dic = json.load(short_word_file)

single_word_dic = read_file(PATH+'correct_teencode_stopword/unidecode_vietnamese_dic.txt')
single_word_dic = [re.sub('\n','', s) for s in single_word_dic]


def replace_one_one(word, dictionary):
    new_word = dictionary.get(word,word)
    return new_word
def correct_vowel(sent, vowel_dictionary):
    '''mapping a`, a\ --> à, ....'''
    words = sent.split()
    pattern = r'[aăâeêuưiyoôơ][.`~?\']'
    sent = ""
    for word in words:
        p = re.search(pattern, word)
        new_word = word
        if p:
            idx = p.span()
            replace_vowel = vowel_dictionary[word[idx[0]]][word[idx[0] + 1]]
            new_word = re.sub(pattern, replace_vowel, new_word)
        sent += new_word + ' '
    return sent

def correct_teencode(sent):
    sent = preprocess(sent)
    sent = correct_vowel(sent, vowel_dic)

    words = sent.split()
    sent = ""
    for word in words:
        new_word = ""
        if word[-1] in [',',';']:
            new_word = replace_one_one(word[:-1], short_word_dic)
            sent += new_word + word[-1]
        else:
            new_word = replace_one_one(word, short_word_dic)
            sent += new_word
        sent += ' '
    return sent[:-1]

In [7]:
data_fixteencode = data_train.copy()
data_fixteencode['Data'] = data_fixteencode['Data'].map(lambda x: correct_teencode(x))

In [8]:
data_fixteencode['Data']

0       mình đã dùng anywhere thế hệ đầu quả là đầy th...
1       quan tâm nhất là độ trễ có cao không dùng thi ...
2       dag xài con cùi bắp 98k pin trâu mỗi tội đánh ...
3       logitech chắc hàng phải tiền triệu trở lên dùn...
4       đang xài con m175 cùi mía nhà xài nhiều chuột ...
                              ...                        
5095    mình mua máy về được 1 ngày mà điện thoại khác...
5096    có bạn nào dùng f1ư không mình dùng m cảm thấy...
5097    dùng ôp mà bộ nhớ 4gb thì không chơi games đượ...
5098    sao tui thích xài hàng ôp mà lựa toàn mấy đứa ...
5099    mới mở hộp rồi mở vào camera mà đã có ảnh chụp...
Name: Data, Length: 5100, dtype: object

In [9]:
data_train = pd.concat([data_fixteencode, data_train])
data_train.shape

(10200, 2)

In [10]:
data_train = data_train.drop_duplicates()
data_train = data_train.dropna()
data_train['Class'].value_counts()

 0    2277
-1    2125
 1    2007
Name: Class, dtype: int64

In [11]:
import codecs
import string
from underthesea import word_tokenize
def create_stopwordlist():
    f = codecs.open(PATH+'correct_teencode_stopword/vlsp_stopwords.txt', encoding='utf-8')
    data = []
    null_data = []
    for i, line in enumerate(f):
        line = repr(line)
        line = line[1:len(line)-3]
        data.append(line)
    return data
stopword_vn = create_stopwordlist()

def tokenize(text):
    text =  text.translate(str.maketrans('', '', string.punctuation))
    return [word for word in word_tokenize(text.lower()) if word not in stopword_vn]
tokens = []
for i in data_train.iloc[:, 1].values:
  tokens.append(tokenize(i))

In [12]:
model = Word2Vec(sentences=tokens, size=400, window=4, min_count=3, workers=4)


In [13]:
# Test
# Find the most simlimar word
sims = model.wv.most_similar('ném', topn=10) 
# The similarity between two word 
a = model.wv.similarity('con chuột', 'ném')
print('\n',sims,'\n',a)


 [('to', 0.999903678894043), ('sợ', 0.9999028444290161), ('đừng', 0.9999024868011475), ('đấy', 0.9999024271965027), ('bảo', 0.9999019503593445), ('kiểu', 0.9999017715454102), ('hai', 0.9999016523361206), ('so sánh', 0.9999013543128967), ('đầu', 0.999901294708252), ('bảo hành', 0.9999008774757385)] 
 0.99954414


In [14]:
labels = data_train.iloc[:, 0].values
reviews = data_train.iloc[:, 1].values

In [15]:
encoded_labels = []

for label in labels:
    if label == -1:
        encoded_labels.append([1,0,0])
    elif label == 0:
        encoded_labels.append([0,1,0])
    else:
        encoded_labels.append([0,0,1])

encoded_labels = np.array(encoded_labels)  

In [16]:
reviews_processed = []
unlabeled_processed = [] 
for review in reviews:
    review_cool_one = ''.join([char for char in review if char not in digits])
    reviews_processed.append(review_cool_one)

In [17]:
#Use PyVi for Vietnamese word tokenizer
word_reviews = []
all_words = []
for review in reviews_processed:
    review = ViTokenizer.tokenize(review.lower())
    word_reviews.append(review.split())
   

In [18]:
EMBEDDING_DIM = 400 # how big is each word vector
MAX_VOCAB_SIZE = 10000 # how many unique words to use (i.e num rows in embedding vector)
MAX_SEQUENCE_LENGTH = 300 # max number of words in a comment to use

In [21]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [22]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, lower=True, char_level=False)
tokenizer.fit_on_texts(word_reviews)
sequences_train = tokenizer.texts_to_sequences(word_reviews)
word_index = tokenizer.word_index


In [23]:
data = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)
labels = encoded_labels

In [24]:
print('Shape of X train and X validation tensor:',data.shape)
print('Shape of label train and validation tensor:', labels.shape)

Shape of X train and X validation tensor: (6409, 300)
Shape of label train and validation tensor: (6409, 3)


In [26]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format(PATH+'model cbow/vi-model-CBOW.bin', binary=True)


vocabulary_size=min(len(word_index)+1,MAX_VOCAB_SIZE)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=MAX_VOCAB_SIZE:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

In [29]:
def scheduler(epoch, lr):
  if epoch < 4:
    return lr
  return lr * tf.math.exp(-0.1)

# CNN

In [30]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# from keras.optimizers import Adam

from keras.models import Model
from keras import regularizers
sequence_length = data.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.3 #0.5

inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
reshape = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)

conv_0 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv2D(num_filters, (filter_sizes[1], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv2D(num_filters, (filter_sizes[2], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_0)
maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1), strides=(1,1))(conv_1)
maxpool_2 = MaxPooling2D((sequence_length - filter_sizes[2] + 1, 1), strides=(1,1))(conv_2)

merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2], axis=1)
flatten = Flatten()(merged_tensor)
reshape = Reshape((3*num_filters,))(flatten)
dropout = Dropout(drop)(reshape)
output = Dense(units=3, activation='softmax',kernel_regularizer=regularizers.l2(0.05))(dropout)

# this creates a model that includes
model = Model(inputs, output)

adam = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.01)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [tf.keras.callbacks.LearningRateScheduler(scheduler),early_stopping]

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 300, 400)     3126000     ['input_2[0][0]']                
                                                                                                  
 reshape_2 (Reshape)            (None, 300, 400, 1)  0           ['embedding[1][0]']              
                                                                                                  
 conv2d_3 (Conv2D)              (None, 298, 1, 100)  120100      ['reshape_2[0][0]']              
                                                                                            

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, stratify=labels)

In [32]:
model.fit(X_train, y_train, epochs=20, batch_size=256, callbacks=callbacks_list, verbose=2, validation_data=(X_test, y_test))

Epoch 1/20
21/21 - 19s - loss: 3.7533 - accuracy: 0.4625 - val_loss: 2.1351 - val_accuracy: 0.4727 - lr: 0.0100 - 19s/epoch - 914ms/step
Epoch 2/20
21/21 - 18s - loss: 1.5087 - accuracy: 0.5697 - val_loss: 1.0493 - val_accuracy: 0.5429 - lr: 0.0100 - 18s/epoch - 844ms/step
Epoch 3/20
21/21 - 18s - loss: 0.9605 - accuracy: 0.6191 - val_loss: 0.8611 - val_accuracy: 0.5975 - lr: 0.0100 - 18s/epoch - 844ms/step
Epoch 4/20
21/21 - 18s - loss: 0.7866 - accuracy: 0.6676 - val_loss: 0.7386 - val_accuracy: 0.6287 - lr: 0.0100 - 18s/epoch - 841ms/step
Epoch 5/20
21/21 - 18s - loss: 0.6763 - accuracy: 0.7041 - val_loss: 0.6509 - val_accuracy: 0.6888 - lr: 0.0090 - 18s/epoch - 847ms/step
Epoch 6/20
21/21 - 18s - loss: 0.6714 - accuracy: 0.7277 - val_loss: 0.6857 - val_accuracy: 0.6669 - lr: 0.0082 - 18s/epoch - 841ms/step
Epoch 7/20
21/21 - 18s - loss: 0.6189 - accuracy: 0.7496 - val_loss: 0.6463 - val_accuracy: 0.6919 - lr: 0.0074 - 18s/epoch - 847ms/step
Epoch 8/20
21/21 - 18s - loss: 0.5590 - a

# LSTM

In [33]:
# from keras.layers import BatchNormalization
from keras.layers import Bidirectional, LSTM
sequence_length = data.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.3 #0.5
hidden_dims = 100

inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
lstm_layer = Bidirectional(LSTM(128,dropout=drop))(embedding)

dense_layer = Dense(hidden_dims*2, activation='sigmoid')(lstm_layer)
dropout= Dropout(0.2)(dense_layer)
output = Dense(units=3, activation='softmax',kernel_regularizer=regularizers.l2(0.05))(dropout)

# this creates a model that includes
model_LSTM = Model(inputs, output)

adam = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.01)
model_LSTM.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model_LSTM.summary()

#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [tf.keras.callbacks.LearningRateScheduler(scheduler),early_stopping]


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 300)]             0         
                                                                 
 embedding (Embedding)       (None, 300, 400)          3126000   
                                                                 
 bidirectional (Bidirectiona  (None, 256)              541696    
 l)                                                              
                                                                 
 dense_2 (Dense)             (None, 200)               51400     
                                                                 
 dropout_2 (Dropout)         (None, 200)               0         
                                                                 
 dense_3 (Dense)             (None, 3)                 603       
                                                           

In [34]:
model_LSTM.fit(X_train, y_train, epochs=20, batch_size=256, callbacks=callbacks_list, verbose=2, validation_data=(X_test, y_test))

Epoch 1/20
21/21 - 40s - loss: 0.6522 - accuracy: 0.6150 - val_loss: 0.5354 - val_accuracy: 0.6388 - lr: 0.0100 - 40s/epoch - 2s/step
Epoch 2/20
21/21 - 36s - loss: 0.3985 - accuracy: 0.7886 - val_loss: 0.4674 - val_accuracy: 0.7231 - lr: 0.0100 - 36s/epoch - 2s/step
Epoch 3/20
21/21 - 36s - loss: 0.2871 - accuracy: 0.8793 - val_loss: 0.4457 - val_accuracy: 0.7441 - lr: 0.0100 - 36s/epoch - 2s/step
Epoch 4/20
21/21 - 35s - loss: 0.2310 - accuracy: 0.9222 - val_loss: 0.4537 - val_accuracy: 0.7566 - lr: 0.0100 - 35s/epoch - 2s/step
Epoch 5/20
21/21 - 35s - loss: 0.2063 - accuracy: 0.9388 - val_loss: 0.4526 - val_accuracy: 0.7590 - lr: 0.0090 - 35s/epoch - 2s/step
Epoch 6/20
21/21 - 35s - loss: 0.1827 - accuracy: 0.9561 - val_loss: 0.4578 - val_accuracy: 0.7566 - lr: 0.0082 - 35s/epoch - 2s/step
Epoch 7/20
21/21 - 35s - loss: 0.1714 - accuracy: 0.9645 - val_loss: 0.4566 - val_accuracy: 0.7660 - lr: 0.0074 - 35s/epoch - 2s/step
Epoch 7: early stopping


# 2 stacked LSTM

In [35]:
from keras.layers import Bidirectional, LSTM
sequence_length = data.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.3 #0.5
hidden_dims = 100

inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
# reshape = Reshape((sequence_length,EMBEDDING_DIM))(embedding)

lstm_3 = Bidirectional(LSTM(128, return_sequences=True))(embedding)
lstm_2 = Bidirectional(LSTM(64))(lstm_3)
# lstm_1 = Bidirectional(LSTM(256, return_sequences=True))(lstm_2)
# lstm_0 = Bidirectional(LSTM(128,dropout=drop))(lstm_1)

# dense_layer = Dense(hidden_dims*2, activation='sigmoid')(lstm_0)
dropout= Dropout(drop)(lstm_2)
output = Dense(units=3, activation='softmax',kernel_regularizer=regularizers.l2(0.05))(dropout)

# this creates a model that includes
model_2LSTM = Model(inputs, output)

adam = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.01)
model_2LSTM.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model_2LSTM.summary()

#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [tf.keras.callbacks.LearningRateScheduler(scheduler),early_stopping]

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 300)]             0         
                                                                 
 embedding (Embedding)       (None, 300, 400)          3126000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 300, 256)         541696    
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 3)                 387 

In [36]:
model_2LSTM.fit(X_train, y_train, epochs=20, batch_size=256, callbacks=callbacks_list, verbose=2, validation_data=(X_test, y_test))

Epoch 1/20
21/21 - 55s - loss: 0.6340 - accuracy: 0.6914 - val_loss: 0.4875 - val_accuracy: 0.7215 - lr: 0.0100 - 55s/epoch - 3s/step
Epoch 2/20
21/21 - 47s - loss: 0.2891 - accuracy: 0.9013 - val_loss: 0.4697 - val_accuracy: 0.7488 - lr: 0.0100 - 47s/epoch - 2s/step
Epoch 3/20
21/21 - 47s - loss: 0.2144 - accuracy: 0.9468 - val_loss: 0.4996 - val_accuracy: 0.7434 - lr: 0.0100 - 47s/epoch - 2s/step
Epoch 4/20
21/21 - 47s - loss: 0.1788 - accuracy: 0.9657 - val_loss: 0.5067 - val_accuracy: 0.7496 - lr: 0.0100 - 47s/epoch - 2s/step
Epoch 5/20
21/21 - 47s - loss: 0.1580 - accuracy: 0.9768 - val_loss: 0.5161 - val_accuracy: 0.7520 - lr: 0.0090 - 47s/epoch - 2s/step
Epoch 6/20
21/21 - 47s - loss: 0.1442 - accuracy: 0.9836 - val_loss: 0.5270 - val_accuracy: 0.7512 - lr: 0.0082 - 47s/epoch - 2s/step
Epoch 6: early stopping


# GRU

In [37]:
# from keras.layers import BatchNormalization
from keras.layers import Bidirectional, GRU
sequence_length = data.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.3 #0.5
hidden_dims = 100

inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
lstm_layer = Bidirectional(GRU(128,dropout=drop))(embedding)

dense_layer = Dense(hidden_dims*2, activation='sigmoid')(lstm_layer)
dropout= Dropout(0.2)(dense_layer)
output = Dense(units=3, activation='softmax',kernel_regularizer=regularizers.l2(0.05))(dropout)

# this creates a model that includes
model_GRU = Model(inputs, output)

adam = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.01)
model_GRU.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model_GRU.summary()

#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [tf.keras.callbacks.LearningRateScheduler(scheduler),early_stopping]


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 300)]             0         
                                                                 
 embedding (Embedding)       (None, 300, 400)          3126000   
                                                                 
 bidirectional_3 (Bidirectio  (None, 256)              407040    
 nal)                                                            
                                                                 
 dense_5 (Dense)             (None, 200)               51400     
                                                                 
 dropout_4 (Dropout)         (None, 200)               0         
                                                                 
 dense_6 (Dense)             (None, 3)                 603       
                                                           

In [38]:
model_GRU.fit(X_train, y_train, epochs=20, batch_size=256, callbacks=callbacks_list, verbose=2, validation_data=(X_test, y_test))

Epoch 1/20
21/21 - 33s - loss: 0.6074 - accuracy: 0.6946 - val_loss: 0.4901 - val_accuracy: 0.7184 - lr: 0.0100 - 33s/epoch - 2s/step
Epoch 2/20
21/21 - 29s - loss: 0.2383 - accuracy: 0.9261 - val_loss: 0.5048 - val_accuracy: 0.7402 - lr: 0.0100 - 29s/epoch - 1s/step
Epoch 3/20
21/21 - 29s - loss: 0.1785 - accuracy: 0.9594 - val_loss: 0.5064 - val_accuracy: 0.7410 - lr: 0.0100 - 29s/epoch - 1s/step
Epoch 4/20
21/21 - 30s - loss: 0.1686 - accuracy: 0.9626 - val_loss: 0.5018 - val_accuracy: 0.7543 - lr: 0.0100 - 30s/epoch - 1s/step
Epoch 5/20
21/21 - 30s - loss: 0.1435 - accuracy: 0.9793 - val_loss: 0.4998 - val_accuracy: 0.7613 - lr: 0.0090 - 30s/epoch - 1s/step
Epoch 5: early stopping


# CNN-LSTM

In [39]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
sequence_length = data.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.3 #0.5

inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
reshape = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)

conv_0 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv2D(num_filters, (filter_sizes[1], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv2D(num_filters, (filter_sizes[2], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)


maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_0)
maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1), strides=(1,1))(conv_1)
maxpool_2 = MaxPooling2D((sequence_length - filter_sizes[2] + 1, 1), strides=(1,1))(conv_2)


merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2], axis=1)
flatten = Flatten()(merged_tensor)
reshape = Reshape((3*num_filters,))(flatten)

x = embedding = embedding_layer(inputs)
lstm_0 = LSTM(128)(x)

cnn_lstm_layer =  concatenate([lstm_0, reshape], axis=1)
dropout = Dropout(drop)(cnn_lstm_layer)
output = Dense(units=3, activation='softmax',kernel_regularizer=regularizers.l2(0.05))(dropout)

# this creates a model that includes
model_cnn_lstm = Model(inputs, output)

adam = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.01)
model_cnn_lstm.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model_cnn_lstm.summary()

#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [tf.keras.callbacks.LearningRateScheduler(scheduler),early_stopping]

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 300, 400)     3126000     ['input_6[0][0]',                
                                                                  'input_6[0][0]']                
                                                                                                  
 reshape_4 (Reshape)            (None, 300, 400, 1)  0           ['embedding[5][0]']              
                                                                                                  
 conv2d_6 (Conv2D)              (None, 298, 1, 100)  120100      ['reshape_4[0][0]']        

In [40]:
model_cnn_lstm.fit(X_train, y_train, epochs=20, batch_size=256, callbacks=callbacks_list, verbose=2, validation_data=(X_test, y_test))

Epoch 1/20


2022-05-01 09:36:52.302886: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 245760000 exceeds 10% of free system memory.
2022-05-01 09:36:53.985464: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 245760000 exceeds 10% of free system memory.
2022-05-01 09:36:55.652530: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 245760000 exceeds 10% of free system memory.
2022-05-01 09:36:57.346746: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 245760000 exceeds 10% of free system memory.
2022-05-01 09:36:59.057763: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 245760000 exceeds 10% of free system memory.


21/21 - 40s - loss: 2.8211 - accuracy: 0.7375 - val_loss: 1.3880 - val_accuracy: 0.7239 - lr: 0.0100 - 40s/epoch - 2s/step
Epoch 2/20
21/21 - 37s - loss: 0.8211 - accuracy: 0.8980 - val_loss: 0.7512 - val_accuracy: 0.7363 - lr: 0.0100 - 37s/epoch - 2s/step
Epoch 3/20
21/21 - 37s - loss: 0.5477 - accuracy: 0.9466 - val_loss: 0.6649 - val_accuracy: 0.7551 - lr: 0.0100 - 37s/epoch - 2s/step
Epoch 4/20
21/21 - 37s - loss: 0.3586 - accuracy: 0.9440 - val_loss: 0.5873 - val_accuracy: 0.7480 - lr: 0.0100 - 37s/epoch - 2s/step
Epoch 5/20
21/21 - 37s - loss: 0.2851 - accuracy: 0.9692 - val_loss: 0.5376 - val_accuracy: 0.7512 - lr: 0.0090 - 37s/epoch - 2s/step
Epoch 6/20
21/21 - 37s - loss: 0.2357 - accuracy: 0.9772 - val_loss: 0.5138 - val_accuracy: 0.7582 - lr: 0.0082 - 37s/epoch - 2s/step
Epoch 7/20
21/21 - 37s - loss: 0.2083 - accuracy: 0.9830 - val_loss: 0.5225 - val_accuracy: 0.7512 - lr: 0.0074 - 37s/epoch - 2s/step
Epoch 8/20
21/21 - 37s - loss: 0.1712 - accuracy: 0.9891 - val_loss: 0.50

# Valid

In [41]:
labels_test = data_test.iloc[:, 0].values
reviews_test = data_test.iloc[:, 1].values

In [42]:
encoded_labels_test = []

for label_test in labels_test:
    if label_test == -1:
        encoded_labels_test.append([1,0,0])
    elif label_test == 0:
        encoded_labels_test.append([0,1,0])
    else:
        encoded_labels_test.append([0,0,1])

encoded_labels_test = np.array(encoded_labels_test)  

In [43]:
reviews_processed_test = []
unlabeled_processed_test = [] 
for review_test in reviews_test:
    review_cool_one = ''.join([char for char in review_test if char not in digits])
    reviews_processed_test.append(review_cool_one)

In [44]:
#Use PyVi for Vietnamese word tokenizer
word_reviews_test = []
all_words = []
for review_test in reviews_processed_test:
    review_test = ViTokenizer.tokenize(review_test.lower())
    word_reviews_test.append(review_test.split())

In [45]:
sequences_test = tokenizer.texts_to_sequences(word_reviews_test)
data_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
labels_test = encoded_labels_test

In [46]:
print('Shape of X train and X validation tensor:',data_test.shape)
print('Shape of label train and validation tensor:', labels_test.shape)

Shape of X train and X validation tensor: (1050, 300)
Shape of label train and validation tensor: (1050, 3)


# CNN

In [51]:
y_pred_cnn = model.predict(data_test)
pred = np.argmax(y_pred_cnn, axis=1)
pred

array([0, 2, 0, ..., 1, 1, 0])

In [52]:
test = np.argmax(labels_test, axis=1)
test

array([0, 0, 0, ..., 1, 1, 1])

In [53]:
from sklearn import metrics
print("Precision:\t", metrics.precision_score(test, pred, average='weighted',))
print("Recall:\t\t", metrics.recall_score(test, pred, average='weighted'))
print("F1:\t\t", metrics.f1_score(test, pred, average='weighted'))
print("Accuracy:\t", metrics.accuracy_score(test, pred))

Precision:	 0.6710960927860953
Recall:		 0.6714285714285714
F1:		 0.6712433781414758
Accuracy:	 0.6714285714285714


In [54]:
print(metrics.classification_report(test, pred))

              precision    recall  f1-score   support

           0       0.67      0.67      0.67       350
           1       0.63      0.62      0.63       350
           2       0.71      0.72      0.71       350

    accuracy                           0.67      1050
   macro avg       0.67      0.67      0.67      1050
weighted avg       0.67      0.67      0.67      1050



# LSTM


In [55]:
score = model_LSTM.evaluate(data_test, labels_test)

33/33 [==============================] - 5s 158ms/step - loss: 0.7578 - accuracy: 0.5857


In [56]:
y_pred_lstm = model_LSTM.predict(data_test)
pred = np.argmax(y_pred_lstm, axis=1)
pred

array([0, 2, 0, ..., 0, 1, 0])

In [57]:
from sklearn import metrics
print("Precision:\t", metrics.precision_score(test, pred, average='weighted',))
print("Recall:\t\t", metrics.recall_score(test, pred, average='weighted'))
print("F1:\t\t", metrics.f1_score(test, pred, average='weighted'))
print("Accuracy:\t", metrics.accuracy_score(test, pred))

Precision:	 0.6179675700322019
Recall:		 0.5857142857142857
F1:		 0.578965797793387
Accuracy:	 0.5857142857142857


In [58]:
print(metrics.classification_report(test, pred))

              precision    recall  f1-score   support

           0       0.51      0.81      0.62       350
           1       0.63      0.43      0.51       350
           2       0.72      0.53      0.61       350

    accuracy                           0.59      1050
   macro avg       0.62      0.59      0.58      1050
weighted avg       0.62      0.59      0.58      1050



# Stack lstm

In [59]:
score = model_2LSTM.evaluate(data_test, labels_test)

33/33 [==============================] - 8s 238ms/step - loss: 0.7425 - accuracy: 0.6305


In [60]:
y_pred_2lstm = model_2LSTM.predict(data_test)
pred = np.argmax(y_pred_2lstm, axis=1)
pred

array([0, 2, 0, ..., 1, 1, 2])

In [61]:
from sklearn import metrics
print("Precision:\t", metrics.precision_score(test, pred, average='weighted',))
print("Recall:\t\t", metrics.recall_score(test, pred, average='weighted'))
print("F1:\t\t", metrics.f1_score(test, pred, average='weighted'))
print("Accuracy:\t", metrics.accuracy_score(test, pred))

Precision:	 0.6348759307853005
Recall:		 0.6304761904761905
F1:		 0.6308252150147337
Accuracy:	 0.6304761904761905


In [62]:
print(metrics.classification_report(test, pred))

              precision    recall  f1-score   support

           0       0.66      0.58      0.62       350
           1       0.57      0.65      0.61       350
           2       0.67      0.67      0.67       350

    accuracy                           0.63      1050
   macro avg       0.63      0.63      0.63      1050
weighted avg       0.63      0.63      0.63      1050



# GRU


In [63]:
score = model_GRU.evaluate(data_test, labels_test)

33/33 [==============================] - 4s 118ms/step - loss: 0.7422 - accuracy: 0.6219


In [64]:
y_pred_gru = model_GRU.predict(data_test)
pred = np.argmax(y_pred_gru, axis=1)
pred

array([0, 2, 0, ..., 1, 1, 2])

In [65]:
print("Precision:\t", metrics.precision_score(test, pred, average='weighted',))
print("Recall:\t\t", metrics.recall_score(test, pred, average='weighted'))
print("F1:\t\t", metrics.f1_score(test, pred, average='weighted'))
print("Accuracy:\t", metrics.accuracy_score(test, pred))

Precision:	 0.6248480171740594
Recall:		 0.621904761904762
F1:		 0.622888664154234
Accuracy:	 0.621904761904762


In [66]:
print(metrics.classification_report(test, pred))

              precision    recall  f1-score   support

           0       0.63      0.62      0.63       350
           1       0.56      0.60      0.58       350
           2       0.69      0.64      0.66       350

    accuracy                           0.62      1050
   macro avg       0.62      0.62      0.62      1050
weighted avg       0.62      0.62      0.62      1050



# CNN-LSTM

In [67]:
score = model_cnn_lstm.evaluate(data_test, labels_test)

33/33 [==============================] - 5s 150ms/step - loss: 0.6973 - accuracy: 0.6352


In [68]:
y_pred_cnn_lstm = model_cnn_lstm.predict(data_test)
pred = np.argmax(y_pred_cnn_lstm, axis=1)
pred

array([1, 2, 0, ..., 1, 1, 2])

In [69]:
print("Precision:\t", metrics.precision_score(test, pred, average='weighted',))
print("Recall:\t\t", metrics.recall_score(test, pred, average='weighted'))
print("F1:\t\t", metrics.f1_score(test, pred, average='weighted'))
print("Accuracy:\t", metrics.accuracy_score(test, pred))

Precision:	 0.635171322749185
Recall:		 0.6352380952380953
F1:		 0.6349293003448344
Accuracy:	 0.6352380952380953


In [70]:
print(metrics.classification_report(test, pred))

              precision    recall  f1-score   support

           0       0.66      0.63      0.64       350
           1       0.58      0.58      0.58       350
           2       0.66      0.69      0.68       350

    accuracy                           0.64      1050
   macro avg       0.64      0.64      0.63      1050
weighted avg       0.64      0.64      0.63      1050

